In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

# Define Which Input Files to Use
The default settings will use the input files recently produced in Step 1) using the notebook `get_eia_demand_data.ipynb`. For those interested in reproducing the exact results included in the repository, you will need to point to the files containing the original `raw` EIA demand data that we querried on 10 Sept 2019.

In [ ]:
merge_with_step1_files = False # used to run step 2 on the most recent files
merge_with_10sept2019_files = True # used to reproduce the documented results
assert((merge_with_step1_files != merge_with_10sept2019_files) and 
       (merge_with_step1_files == True or merge_with_10sept2019_files == True)), "One of these must be true: 'merge_with_step1_files' and 'merge_with_10sept2019_files'"

if merge_with_step1_files:
    input_path = './data'

if merge_with_10sept2019_files:
    # input_path is the path to the downloaded data from Zenodo: https://zenodo.org/record/3517197
    input_path = '/BASE/PATH/TO/ZENODO'
    input_path += '/data/release_2019_Oct/original_eia_files'
    assert(os.path.exists(input_path)), f"You must set the base directory for the Zenodo data {input_path} does not exist"
    # If you did not run step 1, make the /data directory
    if not os.path.exists('./data'):
        os.mkdir('./data')

# Make the output directories

In [ ]:
# Make output directories
out_base = './data/final_results'
if not os.path.exists(out_base):
    os.mkdir(out_base)
    for subdir in ['balancing_authorities', 'regions', 'interconnects', 'contiguous_US']:
        os.mkdir(f"{out_base}/{subdir}")
        print(f"Final results files will be located here: {out_base}/{subdir}")

# Useful functions

In [ ]:
# All 56 balancing authorities that have demand (BA)
def return_all_regions():
    return [
                'AEC', 'AECI', 'CPLE', 'CPLW',
                'DUK', 'FMPP', 'FPC',
                'FPL', 'GVL', 'HST', 'ISNE',
                'JEA', 'LGEE', 'MISO', 'NSB',
                'NYIS', 'PJM', 'SC',
                'SCEG', 'SOCO',
                'SPA', 'SWPP', 'TAL', 'TEC',
                'TVA', 'ERCO',
                'AVA', 'AZPS', 'BANC', 'BPAT',
                'CHPD', 'CISO', 'DOPD',
                'EPE', 'GCPD', 'IID',
                'IPCO', 'LDWP', 'NEVP', 'NWMT',
                'PACE', 'PACW', 'PGE', 'PNM',
                'PSCO', 'PSEI', 'SCL', 'SRP',
                'TEPC', 'TIDC', 'TPWR', 'WACM',
                'WALC', 'WAUW',
                'OVEC', 'SEC',
                ]

# All 54 "usable" balancing authorities (BA) (excludes OVEC and SEC)
# These 2 have significant
# enough reporting problems that we do not impute cleaned data for them.
def return_usable_BAs():
    return [
                'AEC', 'AECI', 'CPLE', 'CPLW',
                'DUK', 'FMPP', 'FPC',
                'FPL', 'GVL', 'HST', 'ISNE',
                'JEA', 'LGEE', 'MISO', 'NSB',
                'NYIS', 'PJM', 'SC',
                'SCEG', 'SOCO',
                'SPA', 'SWPP', 'TAL', 'TEC',
                'TVA', 'ERCO',
                'AVA', 'AZPS', 'BANC', 'BPAT',
                'CHPD', 'CISO', 'DOPD',
                'EPE', 'GCPD', 'IID',
                'IPCO', 'LDWP', 'NEVP', 'NWMT',
                'PACE', 'PACW', 'PGE', 'PNM',
                'PSCO', 'PSEI', 'SCL', 'SRP',
                'TEPC', 'TIDC', 'TPWR', 'WACM',
                'WALC', 'WAUW',
                # 'OVEC', 'SEC',
                ]

# mapping of each balancing authority (BA) to its associated
# U.S. interconnect (IC).
def return_ICs_from_BAs():
    return {
        'EASTERN_IC' : [
                'AEC', 'AECI', 'CPLE', 'CPLW',
                'DUK', 'FMPP', 'FPC',
                'FPL', 'GVL', 'HST', 'ISNE',
                'JEA', 'LGEE', 'MISO', 'NSB',
                'NYIS', 'PJM', 'SC',
                'SCEG', 'SOCO',
                'SPA', 'SWPP', 'TAL', 'TEC',
                'TVA',
                'OVEC', 'SEC',
                ],
        'TEXAS_IC' : [
                'ERCO',
                ],
        'WESTERN_IC' : [
                'AVA', 'AZPS', 'BANC', 'BPAT',
                'CHPD', 'CISO', 'DOPD',
                'EPE', 'GCPD',
                'IID',
                'IPCO', 'LDWP', 'NEVP', 'NWMT',
                'PACE', 'PACW', 'PGE', 'PNM',
                'PSCO', 'PSEI', 'SCL', 'SRP',
                'TEPC', 'TIDC', 'TPWR', 'WACM',
                'WALC', 'WAUW',
                ]
        }

# Defines a mapping between the balancing authorities (BAs)
# and their locally defined region based on EIA naming.
# This uses a json file defining the mapping.
def return_BAs_per_region_map():

    regions = {
            'CENT' : 'Central',
            'MIDW' : 'Midwest',
            'TEN' : 'Tennessee',
            'SE' : 'Southeast',
            'FLA' : 'Florida',
            'CAR' : 'Carolinas',
            'MIDA' : 'Mid-Atlantic',
            'NY' : 'New York',
            'NE' : 'New England',
            'TEX' : 'Texas',
            'CAL' : 'California',
            'NW' : 'Northwest',
            'SW' : 'Southwest'
    }

    rtn_map = {}
    for k, v in regions.items():
        rtn_map[k] = []

    # Load EIA's Blancing Authority Acronym table
    # https://www.eia.gov/realtime_grid/
    df = pd.read_csv('data/balancing_authority_acronyms.csv',
            skiprows=1) # skip first row as it is source info

    # Loop over all rows and fill map
    for idx in df.index:

        # Skip Canada and Mexico
        if df.loc[idx, 'Region'] in ['Canada', 'Mexico']:
            continue

        reg_acronym = ''
        # Get region to acronym
        for k, v in regions.items():
            if v == df.loc[idx, 'Region']:
                reg_acronym = k
                break
        assert(reg_acronym != '')

        rtn_map[reg_acronym].append(df.loc[idx, 'Code'])

    tot = 0
    for k, v in rtn_map.items():
        tot += len(v)
    print(f"Total US48 BAs mapped {tot}.  Recall 11 are generation only.")

    return rtn_map


# Assume the MICE results file is a subset of the original hours
def trim_rows_to_match_length(mice, df):
    mice_start = mice.loc[0, 'date_time']
    mice_end = mice.loc[len(mice.index)-1, 'date_time']
    to_drop = []
    for idx in df.index:
        if df.loc[idx, 'date_time'] != mice_start:
            to_drop.append(idx)
        else: # stop once equal
            break
    for idx in reversed(df.index):
        if df.loc[idx, 'date_time'] != mice_end:
            to_drop.append(idx)
        else: # stop once equal
            break
    
    df = df.drop(to_drop, axis=0)
    df = df.reset_index()
    assert(len(mice.index) == len(df.index))
    return df


# Load balancing authority files already containing the full MICE results.
# Aggregate associated regions into regional, interconnect, or CONUS files.
# Treat 'MISSING' and 'EMPTY' values as zeros when aggregating.
def merge_BAs(region, bas, out_base, folder):
    
    print(region, bas)
    
    # Remove BAs which are generation only as well as SEC and OVEC.
    # See main README regarding SEC and OVEC.
    usable_BAs = return_usable_BAs()
    good_bas = []
    for ba in bas:
        if ba in usable_BAs:
            good_bas.append(ba)
    
    
    first_ba = good_bas.pop()
    master = pd.read_csv(f'{out_base}/balancing_authorities/{first_ba}.csv', na_values=['MISSING', 'EMPTY'])
    master = master.fillna(0)
    
    master = master.drop(['category', 'forecast demand (MW)'], axis=1)
    
    for ba in good_bas:
        df = pd.read_csv(f'{out_base}/balancing_authorities/{ba}.csv', na_values=['MISSING', 'EMPTY'])
        df = df.fillna(0)
        master['raw demand (MW)'] += df['raw demand (MW)']
        master['cleaned demand (MW)'] += df['cleaned demand (MW)']
    
    master.to_csv(f'{out_base}/{folder}/{region}.csv', index=False)
    

# Do both the distribution of balancing authority level results to new BA files
# and generate regional, interconnect, and CONUS aggregate files.
def distribute_MICE_results(raw_demand_file_loc, screening_file, mice_results_csv, out_base):

    # Load screening results
    screening = pd.read_csv(screening_file)
    # Load MICE results
    mice = pd.read_csv(mice_results_csv)
    screening = trim_rows_to_match_length(mice, screening)
    
    # Distribute to single BA results files first
    print("Distribute MICE results per-balancing authority:")
    for ba in return_usable_BAs():
        print(ba)
        df = pd.read_csv(f"{raw_demand_file_loc}/{ba}.csv")
        df = trim_rows_to_match_length(mice, df)
    
        df_out = pd.DataFrame({
            'date_time': df['date_time'],
            'raw demand (MW)': df['demand (MW)'],
            'category': screening[f'{ba}_category'],
            'cleaned demand (MW)': mice[ba],
            'forecast demand (MW)': df['forecast demand (MW)']
        })
        
        
        df_out.to_csv(f'./{out_base}/balancing_authorities/{ba}.csv', index=False)

    # Aggregate balancing authority level results into EIA regions
    print("\nEIA regional aggregation:")
    for region, bas in return_BAs_per_region_map().items():
        merge_BAs(region, bas, out_base, 'regions')
    
    # Aggregate balancing authority level results into CONUS interconnects
    print("\nCONUS interconnect aggregation:")
    for region, bas in return_ICs_from_BAs().items():
        merge_BAs(region, bas, out_base, 'interconnects')
    
    # Aggregate balancing authority level results into CONUS total
    print("\nCONUS total aggregation:")
    merge_BAs('CONUS', return_usable_BAs(), out_base, 'contiguous_US')

# Run the distribution and aggregation

In [ ]:
# The output file generated by Step 2 listing the categories for each time step
screening_file = './data/csv_MASTER.csv'
# The output file generated by Step 3 which runs the MICE algo and has the cleaned demand values
mice_file = 'MICE_output/mean_impute_csv_MASTER.csv'


distribute_MICE_results(input_path, screening_file, mice_file, out_base)

# Test distribution and aggregation
This cell simply checks that the results all add up.

In [ ]:
# Compare each value in the vectors
def compare(vect1, vect2):
    cnt = 0
    clean = True
    for v1, v2 in zip(vect1, vect2):
        if v1 != v2:
            print(f"Error at idx {cnt} {v1} != {v2}")
            clean = False
        cnt += 1
    return clean
    

def test_aggregation(raw_demand_file_loc, screening_file, mice_results_csv, out_base):

    # Load MICE results
    usable_BAs = return_usable_BAs()
    mice = pd.read_csv(mice_results_csv)

    # Sum all result BAs
    tot_imp = np.zeros(len(mice.index))
    for col in mice.columns:
        if col not in usable_BAs:
            continue
        tot_imp += mice[col]

    # Sum Raw
    tot_raw = np.zeros(len(mice.index))
    for ba in return_usable_BAs():
        df = pd.read_csv(f"{raw_demand_file_loc}/{ba}.csv", na_values=['MISSING', 'EMPTY'])
        df = trim_rows_to_match_length(mice, df)
        df = df.fillna(0)

        tot_raw += df['demand (MW)']
    
    # Check BA results distribution
    print("\nBA Distribution:")
    new_tot_raw = np.zeros(len(mice.index))
    new_tot_clean = np.zeros(len(mice.index))
    for ba in return_usable_BAs():
        df = pd.read_csv(f"{out_base}/balancing_authorities/{ba}.csv", na_values=['MISSING', 'EMPTY'])
        df = df.fillna(0)
        new_tot_raw += df['raw demand (MW)']
        new_tot_clean += df['cleaned demand (MW)']
    
    assert(compare(tot_raw, new_tot_raw)), "Error in raw sums."
    assert(compare(tot_imp, new_tot_clean)), "Error in imputed values."
    print("BA Distribution okay!")
    
    
    # Check aggregate balancing authority level results into EIA regions
    print("\nEIA regional aggregation:")
    new_tot_raw = np.zeros(len(mice.index))
    new_tot_clean = np.zeros(len(mice.index))
    for region, bas in return_BAs_per_region_map().items():
        df = pd.read_csv(f"{out_base}/regions/{region}.csv")
        new_tot_raw += df['raw demand (MW)']
        new_tot_clean += df['cleaned demand (MW)']
    
    assert(compare(tot_raw, new_tot_raw)), "Error in raw sums."
    assert(compare(tot_imp, new_tot_clean)), "Error in imputed values."
    print("Regional sums okay!")
    
    # Aggregate balancing authority level results into CONUS interconnects
    print("\nCONUS interconnect aggregation:")
    new_tot_raw = np.zeros(len(mice.index))
    new_tot_clean = np.zeros(len(mice.index))
    for region, bas in return_ICs_from_BAs().items():
        df = pd.read_csv(f"{out_base}/interconnects/{region}.csv")
        new_tot_raw += df['raw demand (MW)']
        new_tot_clean += df['cleaned demand (MW)']
    
    assert(compare(tot_raw, new_tot_raw)), "Error in raw sums."
    assert(compare(tot_imp, new_tot_clean)), "Error in imputed values."
    print("Interconnect sums okay!")
           
           
    # Aggregate balancing authority level results into CONUS total
    print("\nCONUS total aggregation:")
    new_tot_raw = np.zeros(len(mice.index))
    new_tot_clean = np.zeros(len(mice.index))
    df = pd.read_csv(f"{out_base}/contiguous_US/CONUS.csv")
    new_tot_raw += df['raw demand (MW)']
    new_tot_clean += df['cleaned demand (MW)']
    
    assert(compare(tot_raw, new_tot_raw)), "Error in raw sums."
    assert(compare(tot_imp, new_tot_clean)), "Error in imputed values."
    print("CONUS sums okay!")


test_aggregation(input_path, screening_file, mice_file, out_base)